<a href="https://colab.research.google.com/github/Alexandertheasian02/NitroSCAN/blob/Model_training/NitroScan_ModelBuild.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
!pip install pillow

In [ ]:
!pip install numpy

In [ ]:
!pip install opencv-python

In [ ]:
!pip install matplotlib

In [1]:
!pip install SciPy

In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
IMAGE_SIZE=[224,224]

train_dir = '/content/drive/MyDrive/LCC/Splitted_Dataset/train'
val_dir = '/content/drive/MyDrive/LCC/Splitted_Dataset/val'

In [4]:
myResnet = ResNet50(input_shape=IMAGE_SIZE +[3], weights='imagenet' , include_top = False)
print (myResnet.summary())

94765736/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                      

In [5]:
for layer in myResnet.layers:
  layer.trainable=False

In [6]:
Classes = glob('/content/drive/MyDrive/LCC/Splitted_Dataset/train/*')
print(Classes)

['/content/drive/MyDrive/LCC/Splitted_Dataset/train/Level 02', '/content/drive/MyDrive/LCC/Splitted_Dataset/train/Level 03', '/content/drive/MyDrive/LCC/Splitted_Dataset/train/Level 04', '/content/drive/MyDrive/LCC/Splitted_Dataset/train/Level 05']


In [7]:
num_classes=len(Classes)
print(num_classes)

4


In [8]:
PlusFlattenLayer=Flatten()(myResnet.output)
prediction= Dense(num_classes, activation='softmax')(PlusFlattenLayer)
model = Model(inputs = myResnet.input, outputs = prediction)
print (model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [9]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. /255)

training_set= train_datagen.flow_from_directory(train_dir,
                                                target_size=(224,224),
                                                batch_size=32,
                                                class_mode='categorical')

test_set= test_datagen.flow_from_directory(val_dir,
                                           target_size=(224,224),
                                           batch_size=32,
                                           class_mode='categorical')

Found 588 images belonging to 4 classes.
Found 198 images belonging to 4 classes.


In [12]:
result=model.fit(training_set,
                 validation_data=test_set,
                 epochs=50,
                 steps_per_epoch=len(training_set),
                 validation_steps=len(test_set))

Epoch 1/50
19/19 [==============================] - 227s 12s/step - loss: 7.3107 - accuracy: 0.2330 - val_loss: 6.3297 - val_accuracy: 0.2525
Epoch 2/50
19/19 [==============================] - 153s 8s/step - loss: 3.2975 - accuracy: 0.3078 - val_loss: 2.5308 - val_accuracy: 0.3232
Epoch 3/50
19/19 [==============================] - 163s 9s/step - loss: 1.4271 - accuracy: 0.4371 - val_loss: 1.7122 - val_accuracy: 0.4798
Epoch 4/50
19/19 [==============================] - 156s 8s/step - loss: 1.3327 - accuracy: 0.4762 - val_loss: 1.5099 - val_accuracy: 0.4949
Epoch 5/50
19/19 [==============================] - 154s 8s/step - loss: 1.1541 - accuracy: 0.5510 - val_loss: 1.6006 - val_accuracy: 0.4798
Epoch 6/50
19/19 [==============================] - 161s 9s/step - loss: 0.9707 - accuracy: 0.6020 - val_loss: 1.3742 - val_accuracy: 0.5101
Epoch 7/50
19/19 [==============================] - 156s 8s/step - loss: 0.9205 - accuracy: 0.6156 - val_loss: 1.0136 - val_accuracy: 0.6162
Epoch 8/50
1